# How to Use This Dataset
Data Exploration & SQL Practice: Use SQL to join the tables and explore relationships between campaigns, ads, and user engagement.

Market & Strategic Analysis: Evaluate which campaigns, ad types, or targeting strategies yielded the best results.

Cohort & Behavioral Analysis: Track how different user segments (cohorts) behave over time.

Predictive Modeling: Build models to predict user behavior or ad performance based on campaign parameters.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = pd.read_csv('ads.csv')

In [3]:
data.head()

,ad_id,campaign_id,ad_platform,ad_type,target_gender,target_age_group,target_interests
0,1,28,Facebook,Video,Female,35-44,"art, technology"
1,2,33,Facebook,Stories,All,25-34,"travel, photography"
2,3,20,Instagram,Carousel,All,25-34,technology
3,4,28,Facebook,Stories,Female,25-34,news
4,5,24,Instagram,Image,Female,25-34,news


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ad_id             200 non-null    int64 
 1   campaign_id       200 non-null    int64 
 2   ad_platform       200 non-null    object
 3   ad_type           200 non-null    object
 4   target_gender     200 non-null    object
 5   target_age_group  200 non-null    object
 6   target_interests  200 non-null    object
dtypes: int64(2), object(5)
memory usage: 11.1+ KB


In [5]:
df = pd.read_csv(r'ad_events.csv')

In [6]:
df.head()

,event_id,ad_id,user_id,timestamp,day_of_week,time_of_day,event_type
0,1,197,2359b,2025-07-26 00:19:56,Saturday,Night,Like
1,2,51,f9c67,2025-06-15 08:28:07,Sunday,Morning,Share
2,3,46,5b868,2025-06-27 00:40:02,Friday,Night,Impression
3,4,166,3d440,2025-06-05 19:20:45,Thursday,Evening,Impression
4,5,52,68f1a,2025-07-22 08:30:29,Tuesday,Morning,Impression


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   event_id     400000 non-null  int64 
 1   ad_id        400000 non-null  int64 
 2   user_id      400000 non-null  object
 3   timestamp    400000 non-null  object
 4   day_of_week  400000 non-null  object
 5   time_of_day  400000 non-null  object
 6   event_type   400000 non-null  object
dtypes: int64(2), object(5)
memory usage: 21.4+ MB


In [8]:
df1 = pd.read_csv('campaigns.csv')

In [9]:
df1.head()

,campaign_id,name,start_date,end_date,duration_days,total_budget
0,1,Campaign_1_Launch,2025-05-25,2025-07-23,59,24021.32
1,2,Campaign_2_Launch,2025-04-16,2025-07-07,82,79342.41
2,3,Campaign_3_Winter,2025-05-04,2025-06-29,56,14343.25
3,4,Campaign_4_Summer,2025-06-04,2025-08-08,65,45326.60
4,5,Campaign_5_Launch,2025-07-11,2025-08-28,48,68376.69


In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   campaign_id    50 non-null     int64  
 1   name           50 non-null     object 
 2   start_date     50 non-null     object 
 3   end_date       50 non-null     object 
 4   duration_days  50 non-null     int64  
 5   total_budget   50 non-null     float64
dtypes: float64(1), int64(2), object(3)
memory usage: 2.5+ KB


In [ ]:
## merged_df = pd.merge(data, df1, on='ad_id', how='inner')
# "inner" → only matching rows (intersection)
# "left" → all rows from left DataFrame, add matches from right
# "right" → all rows from right DataFrame, add matches from left
# "outer" → union of both DataFrames (all rows, NaN where no match)

In [11]:
data.columns

Index(['ad_id', 'campaign_id', 'ad_platform', 'ad_type', 'target_gender',
       'target_age_group', 'target_interests'],
      dtype='object')

In [12]:
df.columns


Index(['event_id', 'ad_id', 'user_id', 'timestamp', 'day_of_week',
       'time_of_day', 'event_type'],
      dtype='object')

In [13]:
df1.columns

Index(['campaign_id', 'name', 'start_date', 'end_date', 'duration_days',
       'total_budget'],
      dtype='object')

In [14]:
import pandas as pd

# Merge ads with campaigns
ads_campaigns = pd.merge(df1,data, on="campaign_id", how="inner")

# Merge ads+campaigns with events
full_data = pd.merge(ads_campaigns,df, on="ad_id", how="inner")

print(full_data.head())

   campaign_id               name  start_date    end_date  duration_days  \
0            1  Campaign_1_Launch  2025-05-25  2025-07-23             59   
1            1  Campaign_1_Launch  2025-05-25  2025-07-23             59   
2            1  Campaign_1_Launch  2025-05-25  2025-07-23             59   
3            1  Campaign_1_Launch  2025-05-25  2025-07-23             59   
4            1  Campaign_1_Launch  2025-05-25  2025-07-23             59   

   total_budget  ad_id ad_platform  ad_type target_gender target_age_group  \
0      24021.32     56   Instagram  Stories        Female            35-44   
1      24021.32     56   Instagram  Stories        Female            35-44   
2      24021.32     56   Instagram  Stories        Female            35-44   
3      24021.32     56   Instagram  Stories        Female            35-44   
4      24021.32     56   Instagram  Stories        Female            35-44   

  target_interests  event_id user_id            timestamp day_of_week  \
0

In [15]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   campaign_id       400000 non-null  int64  
 1   name              400000 non-null  object 
 2   start_date        400000 non-null  object 
 3   end_date          400000 non-null  object 
 4   duration_days     400000 non-null  int64  
 5   total_budget      400000 non-null  float64
 6   ad_id             400000 non-null  int64  
 7   ad_platform       400000 non-null  object 
 8   ad_type           400000 non-null  object 
 9   target_gender     400000 non-null  object 
 10  target_age_group  400000 non-null  object 
 11  target_interests  400000 non-null  object 
 12  event_id          400000 non-null  int64  
 13  user_id           400000 non-null  object 
 14  timestamp         400000 non-null  object 
 15  day_of_week       400000 non-null  object 
 16  time_of_day       40

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Create a binary target from event_type
full_data["is_click"] = (full_data["event_type"] == "click").astype(int)

# Features (select campaign + ad params, drop IDs/timestamps)
X = full_data[["ad_platform", "ad_type", "target_gender", "target_age_group", "duration_days", "total_budget"]]
X = pd.get_dummies(X)  # encode categorical

y = full_data["is_click"]

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     80000

    accuracy                           1.00     80000
   macro avg       1.00      1.00      1.00     80000
weighted avg       1.00      1.00      1.00     80000



In [17]:
X_train

,duration_days,total_budget,ad_platform_Facebook,ad_platform_Instagram,ad_type_Carousel,ad_type_Image,ad_type_Stories,ad_type_Video,target_gender_All,target_gender_Female,target_gender_Male,target_age_group_18-24,target_age_group_25-34,target_age_group_35-44,target_age_group_All
242245,74,73911.29,True,False,False,False,False,True,False,False,True,False,False,False,True
288918,67,58801.99,False,True,False,True,False,False,True,False,False,False,True,False,False
105103,50,39849.94,False,True,False,True,False,False,True,False,False,False,False,False,True
63504,49,40094.07,True,False,False,False,True,False,False,True,False,True,False,False,False
239180,74,73911.29,True,False,False,False,False,True,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,43,81744.53,False,True,False,False,True,False,False,False,True,False,False,False,True
365838,71,53303.55,True,False,False,False,True,False,False,True,False,False,False,True,False
131932,57,48832.61,True,False,False,False,False,True,False,True,False,False,False,True,False
146867,90,98904.66,True,False,True,False,False,False,False,True,False,True,False,False,False


In [18]:
y_train

242245    0
288918    0
105103    0
63504     0
239180    0
         ..
259178    0
365838    0
131932    0
146867    0
121958    0
Name: is_click, Length: 320000, dtype: int64

In [19]:
from sklearn.metrics import r2_score

In [20]:
r2_score(y_test, y_pred)

1.0

In [21]:
clf.predict([[64,37290.81,True,False,False,False,True,False,False,False,True,False,False,False,True]])

C:\Users\Nayan\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0])